# Plagiarism detection

**Plagiarism** is the use of another's work and ideas as if their are one's original work. For many institutions, it is a habit that is highly discouraged. In the case of an educational institution, we would want to identify cases of plagiarism so that students can receive a stick for such a habit and thereby discouraging them. However, as plagiarism detection techniques get better, so does the methods of avoiding detection. Therefore, in this report we will look at a method that implements a machine learning algorithm so that as the data of users evolves, so will the detection techniques.

## Feature selection

### String matching